In [1]:
import pandas as pd

In [2]:
# Read processed STRING and BioPlex data files.
string = pd.read_csv('../data/string_locs2.csv')
bioplex = pd.read_csv('../data/bioplex_locs2.csv')

# Extract relevant columns from STRING
cols = ['protein1', 'gene1', 'reliability1', 'location1', 'protein2', 'gene2', 'reliability2', 'location2', 'combined_score', 'locations']
string = string[cols]
string = string[string.reliability1.notnull()]
string = string[string.reliability2.notnull()]
max_score = string['combined_score'].max(axis=0)
print(max_score)
string['combined_score'] = string['combined_score'].apply(lambda x: x/max_score)
string['source'] = 'string'

string.shape

999


(698531, 11)

In [3]:
bioplex['source'] = 'bioplex'
bioplex.shape

(73664, 11)

In [4]:
for i in string.index:
    loc = string['locations'][i]
    if "et(" in loc:
        print(loc)
        break

In [5]:
def to_set(x):                      # Get set from location strings
    try:
        x = x[1:-1].replace('"', '')
        x = x.replace("'", "")
        x = x.split(',')
        x = [loc.strip() for loc in x]
        return set(x)
    except:
        return None

In [6]:
# t = pd.merge(string, bioplex, how='inner', on=['protein1', 'protein2']) # Get common datapoints
# print(t.shape)

# locs = list()

# # Count number of common data points that are in agreement. Store the disagreements in a txt file.
# for i in t.index:
#     loc1 = to_set(t['locations_x'][i])
#     loc2 = to_set(t['locations_y'][i])

#     if loc1 == loc2:
#         locs.append(loc1)
#     else:
#         locs.append(loc2)
        
# t['locations'] = locs            
# print(t.head())

In [7]:
df = pd.concat([string, bioplex]).drop_duplicates(subset=['protein1', 'protein2'], keep='last').reset_index(drop=True)

df.head()

,protein1,gene1,reliability1,location1,protein2,gene2,reliability2,location2,combined_score,locations,source
0,ENSP00000001008,ENSG00000004478,Enhanced,"[ Nucleoplasm, Cytosol]",ENSP00000383851,ENSG00000162522,Supported,"[Plasma membrane, Cell Junctions, Nucleopl...",0.182182,[Nucleoplasm],string
1,ENSP00000001008,ENSG00000004478,Enhanced,"[ Nucleoplasm, Cytosol]",ENSP00000383063,ENSG00000180530,Supported,"[Nucleoli, Nucleoplasm, Cytosol]",0.907908,[Nucleoplasm],string
2,ENSP00000001008,ENSG00000004478,Enhanced,"[ Nucleoplasm, Cytosol]",ENSP00000369456,ENSG00000125877,Supported,"[ Nucleoplasm, Nuclear bodies, Cytosol]",0.292292,[Nucleoplasm],string
3,ENSP00000001008,ENSG00000004478,Enhanced,"[ Nucleoplasm, Cytosol]",ENSP00000431512,ENSG00000185122,Supported,"[ Nucleoplasm, Cytosol]",0.955956,"[Cytosol, Nucleoplasm]",string
4,ENSP00000001008,ENSG00000004478,Enhanced,"[ Nucleoplasm, Cytosol]",ENSP00000410452,ENSG00000117632,Supported,[Cytosol],0.235235,[Cytosol],string


In [8]:
df = df.sample(frac=1)
df.to_csv('../data/string_bioplex_trial.csv', index=None)

In [2]:
locs = ['Rods & Rings', 'Aggresome', 'Microtubule ends', 'Cleavage furrow']

df = pd.read_csv('../data/string_bioplex_trial.csv')

FileNotFoundError: [Errno 2] No such file or directory: '../data/string_bioplex_trial.csv'

In [10]:
def remove_locs(x, locs=locs):
    x = to_set(x)
    if x is None:
        return None
    for loc in locs:
        try:
            x.remove(loc)
        except:
            continue
    return x

In [11]:
df['location1'] = df['location1'].apply(lambda x: remove_locs(x))
df['location2'] = df['location2'].apply(lambda x: remove_locs(x))
df['locations'] = df['locations'].apply(lambda x: remove_locs(x))

In [12]:
df.head(1)

,protein1,gene1,reliability1,location1,protein2,gene2,reliability2,location2,combined_score,locations,source
0,ENSP00000377086,ENSG00000170500,Supported,{Nucleoplasm},ENSP00000476117,ENSG00000065268,Supported,{Nucleoplasm},0.291291,{Nucleoplasm},string


In [13]:
from tqdm import tqdm
locations = set()
drop_l = list()

print(df.shape)

for i in tqdm(df.index):
    
    p1, p2 = df['protein1'][i], df['protein2'][i]
    try: 
        b = "ENSP" not in p1 or "ENSP" not in p2
    except:
        drop_l.append(i) 

    l = list(df['locations'][i])
    
    for loc in l:
        locations.add(loc)

df = df.drop(drop_l)
df.shape

100%|██████████| 766080/766080 [00:51<00:00, 14794.75it/s]


(761429, 11)

In [14]:
for loc in locations:
    print(loc)

Lipid droplets
Cell Junctions
Nuclear speckles
Midbody ring
Mitotic spindle
Mitochondria
Intermediate filaments
Midbody
Nuclear membrane
Actin filaments
Vesicles
Lysosomes
Nuclear bodies
Cytokinetic bridge
Plasma membrane
Centrosome
Endosomes
Focal adhesion sites
Centriolar satellite
Peroxisomes
Nucleoli fibrillar center
Cytoplasmic bodies
Golgi apparatus
Endoplasmic reticulum
Microtubules
Cytosol
Nucleoli
Nucleoplasm


In [15]:
len(locations)

28

In [16]:
df.to_csv('../data/string_bioplex_test.csv', index=None)